In [2]:
#dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
#urls to be scraped
news_url = 'https://redplanetscience.com/'
feature_url = 'https://spaceimages-mars.com/'
facts_url = 'https://galaxyfacts-mars.com/'
astro_geo_url = 'https://marshemispheres.com/'

In [4]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\fernb\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [5]:
# visit news site
browser.visit(news_url)

In [9]:
# scrape html soup
html = browser.html
soup = bs(html, 'lxml')

In [10]:
# find first title and body
news_title = soup.find('div', class_="content_title").get_text()
news_p = soup.find('div', class_='article_teaser_body').get_text()

In [11]:
print(news_title)
print(news_p)


NASA's Curiosity Mars Rover Takes a New Selfie Before Record Climb
Along with capturing an image before its steepest ascent ever, the robotic explorer filmed its "selfie stick," or robotic arm, in action.


In [12]:
#visit feature image site
browser.visit(feature_url)

In [13]:
# get url to feature image
links_found = browser.links.find_by_partial_href('feature').first['href']
print(links_found)

https://spaceimages-mars.com/image/featured/mars1.jpg


In [14]:
# go to astro geo site
browser.visit(astro_geo_url)

In [15]:
# get all links to each hemisphere page as list
image_links = browser.find_by_css('.product-item').links.find_by_partial_text('Enhance')

listoflinks = []
for i in range(4):
    listoflinks.append(image_links[i]['href'])

In [16]:
# go to each url in list and get each jpeg url and image title as list of dictionaries
imglist = []
for i in listoflinks:
    browser.visit(i)
    img = browser.links.find_by_text('Sample')
    name = browser.find_by_css('.title').first.value.rstrip('Enhanced').strip()
    dic = {}
    dic['title'] = name
    dic['img_url'] = img['href']
        
    imglist.append(dic)

In [17]:
print(imglist)

[{'title': 'Cerberus Hemisphere', 'img_url': 'https://marshemispheres.com/images/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]


In [18]:
browser.quit()

In [19]:
# pandas read html tables
tables = pd.read_html(facts_url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [31]:
# select first table
df = tables[0]
df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [32]:
#convert back to html table
html_table = df.to_html(index=False)

